In [2]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

In [3]:
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
product_data.head()

,market_ids,city_ids,quarter,product_ids,firm_ids,brand_ids,shares,prices,sugar,mushy,...,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19
0,C01Q1,1,1,F1B04,1,4,0.012417,0.072088,2,1,...,2.116358,-0.154708,-0.005796,0.014538,0.126244,0.067345,0.068423,0.034800,0.126346,0.035484
1,C01Q1,1,1,F1B06,1,6,0.007809,0.114178,18,1,...,-7.374091,-0.576412,0.012991,0.076143,0.029736,0.087867,0.110501,0.087784,0.049872,0.072579
2,C01Q1,1,1,F1B07,1,7,0.012995,0.132391,4,1,...,2.187872,-0.207346,0.003509,0.091781,0.163773,0.111881,0.108226,0.086439,0.122347,0.101842
3,C01Q1,1,1,F1B09,1,9,0.005770,0.130344,3,0,...,2.704576,0.040748,-0.003724,0.094732,0.135274,0.088090,0.101767,0.101777,0.110741,0.104332
4,C01Q1,1,1,F1B11,1,11,0.017934,0.154823,12,0,...,1.261242,0.034836,-0.000568,0.102451,0.130640,0.084818,0.101075,0.125169,0.133464,0.121111


In [4]:
logit_formulation = pyblp.Formulation('prices', absorb='C(product_ids)')
logit_formulation

prices + Absorb[C(product_ids)]

In [7]:
problem = pyblp.Problem(logit_formulation, product_data)
problem

PatsyError: Error evaluating factor: AttributeError: `np.unicode_` was removed in the NumPy 2.0 release. Use `np.str_` instead.
    C(product_ids) - 1
    ^^^^^^^^^^^^^^

In [8]:
logit_results = problem.solve()
logit_results

NameError: name 'problem' is not defined

In [9]:
def solve_nl(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl_formulation = pyblp.Formulation('0 + prices')
    problem = pyblp.Problem(nl_formulation, df)
    return problem.solve(rho=0.7)

In [10]:
df1 = product_data.copy()
df1['nesting_ids'] = 1
nl_results1 = solve_nl(df1)
nl_results1

Problem Results Summary:
GMM   Objective    Projected    Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  --------  -------  ----------------  -----------------
 2    +2.0E+02     +3.5E-10     +1.1E+04     0         +2.0E+09          +3.0E+04     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:01       Yes          3             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +9.8E-01 
(+1.4E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -1.2E+00 
(+4.0E-01)

In [11]:
nl_results1.problem

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
94   2256   5    1     21    1 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices

In [12]:
df2 = product_data.copy()
df2['nesting_ids'] = df2['mushy']
nl_results2 = solve_nl(df2)
nl_results2

Problem Results Summary:
GMM   Objective    Projected    Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  --------  -------  ----------------  -----------------
 2    +6.9E+02     +1.6E-09     +5.6E+03     0         +5.1E+08          +2.0E+04     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:01       Yes          3             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +8.9E-01 
(+1.9E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -7.8E+00 
(+4.8E-01)

In [13]:
nl_results1.beta[0] / (1 - nl_results1.rho)

array([[-67.39338888]])

In [14]:
nl_results2.beta[0] / (1 - nl_results2.rho)

array([[-72.27074638]])

In [15]:
def solve_nl2(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['group_share'] = groups['shares'].transform(np.sum)
    df['within_share'] = df['shares'] / df['group_share']
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl2_formulation = pyblp.Formulation('0 + prices + log(within_share)')
    problem = pyblp.Problem(nl2_formulation, df.drop(columns=['nesting_ids']))
    return problem.solve()

In [16]:
nl2_results1 = solve_nl2(df1)
nl2_results1

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6680\3672487175.py:3: FutureWarning: The provided callable <function sum at 0x000002192FDA91C0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df['group_share'] = groups['shares'].transform(np.sum)


Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +2.0E+02      0         +2.1E+09          +1.1E+04     

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices    log(within_share)
----------  -----------------
 -1.0E+00       +9.9E-01     
(+2.4E-01)     (+7.9E-03)    

In [17]:
nl2_results2 = solve_nl2(df2)
nl2_results2

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6680\3672487175.py:3: FutureWarning: The provided callable <function sum at 0x000002192FDA91C0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df['group_share'] = groups['shares'].transform(np.sum)


Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +7.0E+02      0         +5.5E+08          +7.7E+03     

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices    log(within_share)
----------  -----------------
 -6.8E+00       +9.3E-01     
(+2.9E-01)     (+1.1E-02)    

In [18]:
nl2_results1.beta[0] / (1 - nl2_results1.beta[1])

array([-86.37368445])

In [19]:
nl2_results2.beta[0] / (1 - nl2_results2.beta[1])

array([-100.14496891])